In [10]:
import pandas as pd
import numpy as np
import re
import datetime
import scipy.stats as st

from ib_insync import *

# 01. CONNECT
util.startLoop()

ib=IB()

ib.connect('127.0.0.1', 4002, clientId=2) # kashi IBG paper-trade

<IB connected to 127.0.0.1:4002 clientId=2>

In [12]:
# 02. SETUP THE LIMITS

# Standard Deviation limits
call_probability = 0.97  # for Calls
put_probability = 0.95   # for Puts

call_sd = st.norm.ppf(1-(1-call_probability)/2)
put_sd = st.norm.ppf(1-(1-put_probability)/2)

# Days for maximum standard deviation
dte_max = 365

In [1]:
# 03. PREPARE THE SCRIP LIST

# Download cboe weeklies to a dataframe
dls = "http://www.cboe.com/publish/weelkysmf/weeklysmf.xls"
raw_df = pd.read_excel(dls)

# Extract expiration dates by type
df1 = raw_df.iloc[:9, 2:]

df2 = df1.T

# Types of expiry columns
expiry_type = ['Std_Week', 'Expanded_Week', 'EOW', 'SPX_EOW', 'XSP_Wed', 'Mon_Wed', 'Mon', 'Wed', 'VIX']
df2.columns = expiry_type

# Drop the first row
df2.drop('Unnamed: 2', inplace=True)

# Make an expiries dataframe with a new index
expiries = df2.reset_index().drop('index', 1)

# Extract Tickers and the Expiry Markers [x]

first_row = 12

# Select only columns of interest
a = raw_df.iloc[first_row:, 0]
b = raw_df.iloc[first_row:, 2:4]
c = raw_df.iloc[first_row:, 5:12]
tickers = pd.concat((pd.DataFrame(a), b, c), 1)

# Rename columns
col_names = ['Ticker', 'Desc', 'Type'] + expiry_type[:3] + expiry_type[5:9]
tickers.columns = col_names

# Remove non-null tickers
tickers = tickers.loc[tickers.Ticker.notnull(), :]

# Remove strange characters from tickers
pattern = re.compile('[\W_]+')  # Matches any non-word character and underscore
pattern.sub('', tickers.Ticker[14])
tickers.Ticker = tickers.Ticker.apply(lambda x: pattern.sub('', x))

# Clean up to reflect Index Funds in Type
tickers.loc[tickers.Type.str.contains('Index', na=False), 'Type'] = 'Index'

# Get target columns for each ticker
scriplist = []

for target_col in list(tickers)[3:]:
    ticker_cols = ['Ticker', 'Desc', 'Type']

    # Gets the tickers for target_col
    tick = tickers.loc[(tickers[target_col]) == 'X', ticker_cols]

    # Get the cleaned expiries
    exp = expiries.loc[:, target_col]
    exp = pd.to_datetime(exp, errors='coerce').dropna()

    # Repeat Expiries for the ticker
    scrips = pd.DataFrame(np.repeat(tick.values, len(exp), axis=0),
                          columns=tick.columns,
                          index=np.tile(exp, len(tick))).rename_axis('Expiry').reset_index()

    # Appends the scrips for each expiry type
    scriplist.append(scrips)

# Drops duplicates and makes a neat dataframe!
scriplist = pd.concat(scriplist).drop_duplicates(keep='first').reset_index(drop=True)

,Expiry,Ticker,Desc,Type
0,2018-08-24,OEX,S&P 100 Index (American style),Index
1,2018-08-24,XEO,S&P 100 Index (European style),Index
2,2018-08-24,DJX,Dow Jones Industrial Average,Index
3,2018-08-24,AMJ,clx,ETF
4,2018-08-24,ASHR,Xtrackers Harvest CSI 300 China A-Shares ETF*,ETF
5,2018-08-24,DIA,SPDR DJIA ETF Trust*,ETF
6,2018-08-24,DUST,Direxion Daily Gold Miners Index Bear 3X Shares*,ETF
7,2018-08-24,DXJ,WisdomTree Japan Hedged Equity Fund*,ETF
8,2018-08-24,EEM,iShares MSCI Emerging Markets ETF*,ETF
9,2018-08-24,EFA,iShares MSCI EAFE ETF*,ETF


ERROR:ib_insync.wrapper:Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
ERROR:ib_insync.wrapper:Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained.
ERROR:ib_insync.wrapper:Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
ERROR:ib_insync.client:Peer closed connection
